# VQE Prototype
A simple example for a VQE utilizing the full qiskit library and a simple test hamiltonian following <a href = https://github.com/DavitKhach/quantum-algorithms-tutorials/blob/master/variational_quantum_eigensolver.ipynb>this tutorial</a>.
. The Hamiltonian will be described as the Pauli string
$$H = 0.6 * IX + 0.8 * IZ + 0.2 * XY $$
where the multiplication without any symol in between denotes the Kronecker Product.

System Settings: Since Qiskit Aqua has been deprecated, this example may not run on the newest versions of python and qiskit. This example was created using python 3.9 and qiskit 0.31. Newer versions may not be compatible.

In [14]:
# Imports
import numpy as np
from random import random
from scipy.optimize import minimize


from qiskit import *
from qiskit.circuit.library.standard_gates import U2Gate
from qiskit.aqua.operators import WeightedPauliOperator
from qiskit.aqua.algorithms import NumPyEigensolver

In [2]:
def hamiltonian_operator(a, b, c):
    """
    Creates a*I + b*Z + c*X + d*Y pauli sum
    that will be our Hamiltonian operator.

    """
    # the tensor product is specified by the string concatenation of the pauli labels
    # coefficients are then additionally added

    pauli_dict = {
        'paulis': [{"coeff": {"imag": 0.0, "real": a}, "label": "IX"},
                   {"coeff": {"imag": 0.0, "real": b}, "label": "IZ"},
                   {"coeff": {"imag": 0.0, "real": c}, "label": "XY"},
                   ]
    }
    return WeightedPauliOperator.from_dict(pauli_dict)

In [3]:
# coefficient definition
a = 0.6
b = 0.8
c = 0.2

H = hamiltonian_operator(a, b, c)

/home/christoph/anaconda3/envs/qc/lib/python3.9/site-packages/qiskit/aqua/operators/legacy/weighted_pauli_operator.py:582: DeprecationWarning: The package qiskit.aqua.operators is deprecated. It was moved/refactored to qiskit.opflow (pip install qiskit-terra). For more information see <https://github.com/Qiskit/qiskit-aqua/blob/main/README.md#migration-guide>
  return cls(paulis=paulis)


In [4]:
#confirm that H is indeed acting on two qubits
print(H)

Representation: paulis, qubits: 2, size: 3


In [5]:
# use numpy eigensolver to exactly determine the eigenstates of the hamiltonion
# note that there is no quantum speedup in this
exact_result = NumPyEigensolver(H).run()
reference_energy = min(np.real(exact_result.eigenvalues))
print('The exact ground state energy is: {}'.format(reference_energy))

The exact ground state energy is: -1.0198039027185573


/home/christoph/anaconda3/envs/qc/lib/python3.9/site-packages/qiskit/aqua/algorithms/eigen_solvers/eigen_solver.py:36: DeprecationWarning: The package qiskit.aqua.algorithms.eigen_solvers is deprecated. It was moved/refactored to qiskit.algorithms.eigen_solvers (pip install qiskit-terra). For more information see <https://github.com/Qiskit/qiskit-aqua/blob/main/README.md#migration-guide>
  warn_package('aqua.algorithms.eigen_solvers',


The energy is negative as it is a bound energy state.

## Creating the Ansatz state:
As Ansatz state, we start with the method proposed by the tutorial, i.e. in performing two rotations Rx, Ry onto each of our qubits, specified by parameters t1 - t4. It can easily be shown on the Bloch sphere that two rotations about different axes are universal i.e. each possible state on the Bloch sphere can be reached.

Later this will be changed in favour of the k-UCC Ansatz state.

In [22]:
# creates a circuit doing the preparation on an initial qubit in the |0> state
def universal_prep_state(circuit, params):

    assert(len(params) == 4)
    q1 = circuit.qubits[0]
    q2 = circuit.qubits[1]

    # rotations on first qubit
    circuit.rx(params[0], q1)
    circuit.ry(params[1], q1)

    # rotations on second qubit
    circuit.rx(params[2], q2)
    circuit.ry(params[3], q2)

## Creating the circuit

In [17]:
def vqe_circuit(parameters, measure):
    """
    Creates a device ansatz circuit for optimization.
    :param parameters_array: list of parameters for constructing ansatz state that should be optimized.
    :param measure: measurement type. E.g. 'Z' stands for Z measurement.
    :return: quantum circuit.
    """
    q1 = QuantumRegister(1)
    q2 = QuantumRegister(2)
    c = ClassicalRegister(1)
    circuit = QuantumCircuit(q1, q2, c)

    # quantum state preparation
    circuit = universal_prep_state(circuit, parameters)

    # measurement in differing basis
    if measure == 'Z':
        circuit.measure(q1[0], q2[0], c[0])
    elif measure == 'X':
        circuit.u2(0, np.pi, q1[0])
        circuit.u2(0, np.pi, q2[0])
        circuit.measure(q1[0], q2[0], c[0])

    elif measure == 'Y':
        circuit.u2(0, np.pi/2, q1[0])
        circuit.u2(0, np.pi/2, q2[0])
        circuit.measure(q1[0], q2[0], c[0])
    else:
        raise ValueError('Not valid input for measurement: input should be "X" or "Y" or "Z"')

    return circuit


In [8]:
# quantum module is the function governing the vqe measurements on top of the circuit creation
def quantum_module(parameters, measure):
    # measure
    if measure == 'I':
        return 1
    elif measure == 'Z':
        circuit = vqe_circuit(parameters, 'Z')
    elif measure == 'X':
        circuit = vqe_circuit(parameters, 'X')
    elif measure == 'Y':
        circuit = vqe_circuit(parameters, 'Y')
    else:
        raise ValueError('Not valid input for measurement: input should be "I" or "X" or "Z" or "Y"')

    # many iterations
    shots = 8192
    # execute simulation
    backend = BasicAer.get_backend('qasm_simulator')
    job = execute(circuit, backend, shots=shots)
    result = job.result()
    counts = result.get_counts()

    # expectation value estimation from counts
    expectation_value = 0
    for measure_result in counts:
        sign = +1
        if measure_result == '1':
            sign = -1
        expectation_value += sign * counts[measure_result] / shots

    return expectation_value

In [9]:
def pauli_operator_to_dict(pauli_operator):
    """
    from WeightedPauliOperator return a dict storing the respective weights
    :param pauli_operator: qiskit's WeightedPauliOperator
    :return: a dict in the desired form.
    """
    d = pauli_operator.to_dict()
    paulis = d['paulis']
    paulis_dict = {}

    for x in paulis:
        label = x['label']
        coeff = x['coeff']['real']
        paulis_dict[label] = coeff

    return paulis_dict

# create the necessary Hamiltonian decomposition
pauli_dict = pauli_operator_to_dict(H)



In [11]:
# confirm that everything went right
print(pauli_dict)

{'IX': 0.6, 'IZ': 0.8, 'XY': 0.2}


In [19]:
def vqe(parameters):

    # quantum_modules for each of the measurements and then add the results
    # multiply weight by the expectation values of the decomposed circuits to
    # get the correct result for the whole Hamiltonian
    quantum_module_I = pauli_dict['IX'] * quantum_module(parameters, 'I')
    quantum_module_Z = pauli_dict['IZ'] * quantum_module(parameters, 'Z')
    quantum_module_X = pauli_dict['XY'] * quantum_module(parameters, 'X')

    # summing the measurement results
    classical_adder = quantum_module_I + quantum_module_Z + quantum_module_X

    return classical_adder

In [23]:

parameters_array = np.array([np.pi, np.pi, np.pi, np.pi])
tol = 1e-3 # tolerance for optimization precision.

vqe_result = minimize(vqe, parameters_array, method="Powell", tol=tol)
print('The exact ground state energy is: {}'.format(reference_energy))
print('The estimated ground state energy from VQE algorithm is: {}'.format(vqe_result.fun))



AttributeError: 'NoneType' object has no attribute 'measure'